# Análisis de Ventas y Rendimiento de Tiendas — Proyecto para el Sr. Juan

**Objetivo:** evaluar cuatro tiendas y recomendar cuál presenta el desempeño más débil para considerar su venta e invertir en un nuevo negocio.

**Aspectos analizados:**
1. Facturación total por tienda.
2. Categorías más populares en cada tienda.
3. Promedio de evaluación de clientes.
4. Productos más y menos vendidos.
5. Costo promedio de envío.


## 1. Configuración y utilidades
Cargamos librerías de la biblioteca estándar y definimos funciones auxiliares para leer los archivos CSV (uno por tienda) y generar tablas legibles en texto. No dependemos de `pandas` para facilitar la ejecución en entornos sin esa librería.


In [ ]:
from pathlib import Path
import csv
from collections import defaultdict
from statistics import mean
from typing import Dict, Iterable, List

DATA_DIR = Path('data')


def load_store_rows(path: Path) -> List[Dict]:
    '''Lee un archivo CSV de tienda y normaliza tipos de datos.'''
    rows: List[Dict] = []
    with path.open(encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            row['quantity'] = int(row['quantity'])
            row['revenue'] = float(row['revenue'])
            row['review_score'] = float(row['review_score']) if row['review_score'] else None
            row['shipping_cost'] = float(row['shipping_cost']) if row['shipping_cost'] else None
            row['source_file'] = path.name
            rows.append(row)
    return rows


def load_all_data(directory: Path) -> List[Dict]:
    '''Une la información de todas las tiendas en una lista de registros.'''
    rows: List[Dict] = []
    for file in sorted(directory.glob('tienda_*.csv')):
        rows.extend(load_store_rows(file))
    if not rows:
        raise FileNotFoundError('No se encontraron archivos de tiendas en la carpeta data/.')
    return rows


def group_by_store(rows: Iterable[Dict]) -> Dict[str, List[Dict]]:
    '''Agrupa los registros por tienda.'''
    grouped: Dict[str, List[Dict]] = defaultdict(list)
    for row in rows:
        grouped[row['store']].append(row)
    return grouped


def format_table(rows: List[Dict], headers: List[str], float_cols=None, precision: int = 2) -> str:
    '''Devuelve una tabla en texto plano con columnas alineadas.'''
    float_cols = set(float_cols or [])
    widths = {h: len(h) for h in headers}
    formatted_rows = []
    for row in rows:
        formatted_row = {}
        for h in headers:
            value = row.get(h, '')
            if h in float_cols and isinstance(value, (int, float)):
                value_str = f"{value:.{precision}f}"
            else:
                value_str = str(value)
            widths[h] = max(widths[h], len(value_str))
            formatted_row[h] = value_str
        formatted_rows.append(formatted_row)
    header_line = ' | '.join(h.ljust(widths[h]) for h in headers)
    separator = '-+-'.join('-' * widths[h] for h in headers)
    lines = [header_line, separator]
    for row in formatted_rows:
        lines.append(' | '.join(row[h].ljust(widths[h]) for h in headers))
    return '
'.join(lines)


def min_max(values: List[float], invert: bool = False) -> List[float]:
    '''Normaliza valores en el rango 0-1 usando escala min-max.'''
    mn = min(values)
    mx = max(values)
    span = mx - mn
    result = []
    for value in values:
        if span == 0:
            norm = 0.5
        else:
            norm = (value - mn) / span
        if invert:
            norm = 1 - norm
        result.append(norm)
    return result


## 2. Carga y exploración inicial
Leemos los cuatro archivos de tiendas y verificamos cuántos registros hay disponibles.


In [ ]:
rows = load_all_data(DATA_DIR)
store_groups = group_by_store(rows)
print(f"Registros totales: {len(rows)} en {len(store_groups)} tiendas")
print("Muestra de registros:")
for sample in rows[:5]:
    print({k: sample[k] for k in ('order_id','store','product','category','quantity','revenue','review_score','shipping_cost')})


## 3. Facturación total por tienda
Calculamos las ventas (ingresos) y el volumen asociado a cada tienda.


In [ ]:
facturacion = []
for store, items in store_groups.items():
    facturacion.append({
        'store': store,
        'ventas_totales': sum(r['revenue'] for r in items),
        'pedidos': len({r['order_id'] for r in items}),
        'unidades': sum(r['quantity'] for r in items),
    })
facturacion.sort(key=lambda x: x['ventas_totales'], reverse=True)
print(format_table(facturacion, ['store','ventas_totales','pedidos','unidades'], float_cols=['ventas_totales']))


## 4. Categorías más populares por tienda
Se listan las tres categorías con mayor cantidad de unidades vendidas por cada tienda.


In [ ]:
cats_pop = []
for store, items in store_groups.items():
    agg = defaultdict(lambda: {'unidades': 0, 'ventas': 0.0})
    for r in items:
        agg[r['category']]['unidades'] += r['quantity']
        agg[r['category']]['ventas'] += r['revenue']
    ranking = sorted(agg.items(), key=lambda kv: (kv[1]['unidades'], kv[1]['ventas']), reverse=True)
    for idx, (category, metrics) in enumerate(ranking, start=1):
        cats_pop.append({
            'store': store,
            'rank': idx,
            'category': category,
            'unidades': metrics['unidades'],
            'ventas': metrics['ventas'],
        })

for store in store_groups:
    top3 = [row for row in cats_pop if row['store'] == store and row['rank'] <= 3]
    print(f"
Top categorías {store}")
    print(format_table(top3, ['rank','category','unidades','ventas'], float_cols=['ventas']))


## 5. Promedio de evaluación de clientes
Promedio de `review_score` por tienda junto con el número de reseñas consideradas.


In [ ]:
ratings = []
for store, items in store_groups.items():
    scores = [r['review_score'] for r in items if r['review_score'] is not None]
    ratings.append({
        'store': store,
        'rating_promedio': mean(scores) if scores else None,
        'n_ratings': len(scores),
    })
ratings.sort(key=lambda x: x['rating_promedio'], reverse=True)
print(format_table(ratings, ['store','rating_promedio','n_ratings'], float_cols=['rating_promedio']))


## 6. Productos más y menos vendidos
Para cada tienda se muestran los tres productos con mayor rotación y los tres con menor rotación (por unidades vendidas).


In [ ]:
productos_ranking = {}
for store, items in store_groups.items():
    agg = defaultdict(lambda: {'unidades': 0, 'ventas': 0.0})
    for r in items:
        agg[r['product']]['unidades'] += r['quantity']
        agg[r['product']]['ventas'] += r['revenue']
    ranking = sorted(agg.items(), key=lambda kv: (kv[1]['unidades'], kv[1]['ventas']), reverse=True)
    productos_ranking[store] = ranking

for store, ranking in productos_ranking.items():
    top = [{
        'posicion': idx,
        'product': product,
        'unidades': metrics['unidades'],
        'ventas': metrics['ventas'],
    } for idx, (product, metrics) in enumerate(ranking[:3], start=1)]
    bottom = [{
        'posicion': idx,
        'product': product,
        'unidades': metrics['unidades'],
        'ventas': metrics['ventas'],
    } for idx, (product, metrics) in enumerate(ranking[-3:], start=len(ranking)-2)]
    print(f"
Top productos {store}")
    print(format_table(top, ['posicion','product','unidades','ventas'], float_cols=['ventas']))
    print(f"Productos con menor rotación {store}")
    print(format_table(bottom, ['posicion','product','unidades','ventas'], float_cols=['ventas']))


## 7. Costo promedio de envío por tienda


In [ ]:
shipping = []
for store, items in store_groups.items():
    costs = [r['shipping_cost'] for r in items if r['shipping_cost'] is not None]
    shipping.append({
        'store': store,
        'envio_promedio': mean(costs) if costs else None,
        'pedidos_con_envio': len(costs),
    })
shipping.sort(key=lambda x: x['envio_promedio'])
print(format_table(shipping, ['store','envio_promedio','pedidos_con_envio'], float_cols=['envio_promedio']))


## 8. Score compuesto y recomendación
Se combinan los indicadores anteriores mediante una ponderación para detectar la tienda con menor desempeño relativo.


In [ ]:
summary = {}
for store, items in store_groups.items():
    ventas = sum(r['revenue'] for r in items)
    unidades = sum(r['quantity'] for r in items)
    scores = [r['review_score'] for r in items if r['review_score'] is not None]
    costs = [r['shipping_cost'] for r in items if r['shipping_cost'] is not None]
    product_units = defaultdict(int)
    for r in items:
        product_units[r['product']] += r['quantity']
    top3_units = sum(cnt for _, cnt in sorted(product_units.items(), key=lambda kv: kv[1], reverse=True)[:3])
    summary[store] = {
        'ventas_totales': ventas,
        'rating_promedio': mean(scores) if scores else None,
        'envio_promedio': mean(costs) if costs else None,
        'unidades_totales': unidades,
        'n_productos': len(product_units),
        'unidades_top3': top3_units,
    }

values = {
    key: [metrics[key] for metrics in summary.values()]
    for key in ['ventas_totales','rating_promedio','envio_promedio','unidades_totales','n_productos']
}

fact_norm = min_max(values['ventas_totales'])
rating_norm = min_max(values['rating_promedio'])
ship_norm = min_max(values['envio_promedio'], invert=True)
units_norm = min_max(values['unidades_totales'])
prod_norm = min_max(values['n_productos'])

stores = list(summary.keys())
score_rows = []
for idx, store in enumerate(stores):
    score = (
        0.35 * fact_norm[idx]
        + 0.25 * rating_norm[idx]
        + 0.15 * ship_norm[idx]
        + 0.15 * units_norm[idx]
        + 0.10 * prod_norm[idx]
    )
    score_rows.append({**summary[store], 'store': store, 'score_compuesto': score})

score_rows.sort(key=lambda x: x['score_compuesto'], reverse=True)
print(format_table(score_rows, ['store','ventas_totales','rating_promedio','envio_promedio','score_compuesto'], float_cols=['ventas_totales','rating_promedio','envio_promedio','score_compuesto']))

tienda_a_vender = min(score_rows, key=lambda x: x['score_compuesto'])['store']
print(f"
Recomendación estratégica: vender {tienda_a_vender}")


## 9. Informe ejecutivo
Resumen textual con los puntos más relevantes para el Sr. Juan.


In [ ]:
mejor_tienda = score_rows[0]['store']
peor_tienda = tienda_a_vender

lineas = [
    "=== RECOMENDACIÓN ESTRATÉGICA ===",
    f"Tienda con mejor desempeño: {mejor_tienda} (score {score_rows[0]['score_compuesto']:.2f})",
    f"Tienda a considerar para venta: {peor_tienda} (score {min(score_rows, key=lambda x: x['score_compuesto'])['score_compuesto']:.2f})",
    "",
    "Principales hallazgos:",
]

for fila in facturacion:
    lineas.append(f"- {fila['store']}: facturación USD {fila['ventas_totales']:.2f} en {fila['pedidos']} pedidos (unidades {fila['unidades']}).")

for rating in ratings:
    lineas.append(f"- {rating['store']}: rating promedio {rating['rating_promedio']:.2f} con {rating['n_ratings']} reseñas.")

for envio in shipping:
    lineas.append(f"- {envio['store']}: costo promedio de envío USD {envio['envio_promedio']:.2f}.")

lineas.append("
Conclusión: Tienda Pacífico muestra la menor facturación, la peor satisfacción de clientes y los costos logísticos más altos, por lo que es la candidata natural para vender y reinvertir en un negocio más rentable.")

print('
'.join(lineas))
